In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
import os

# Define dataset path
train_dir = 'dataset/training_set'
test_dir = 'dataset/test_set'

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values to [0, 1]
    shear_range=0.2,             # Random shear transformation
    zoom_range=0.2,              # Random zoom transformation
    horizontal_flip=True         # Random horizontal flip
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load Training Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

# Load Test Data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

# Print class labels
print("Class Indices:", train_generator.class_indices)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Class Indices: {'cats': 0, 'dogs': 1}


In [2]:
IMG_SIZE = 128

# Build CNN model
model = Sequential()

# First convolutional block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output to feed into the dense layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(256, activation='relu'))

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification (0 = Cat, 1 = Dog)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

C:\Users\vvale\anaconda3\envs\python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,516,289 (24.86 MB)

 Trainable params: 6,516,289 (24.86 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train Model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 174s 686ms/step - accuracy: 0.5220 - loss: 0.7226 - val_accuracy: 0.6050 - val_loss: 0.6707
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 177s 702ms/step - accuracy: 0.6473 - loss: 0.6290 - val_accuracy: 0.6639 - val_loss: 0.6022
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 170s 678ms/step - accuracy: 0.6926 - loss: 0.5757 - val_accuracy: 0.7598 - val_loss: 0.5090
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 159s 634ms/step - accuracy: 0.7238 - loss: 0.5319 - val_accuracy: 0.7662 - val_loss: 0.4941
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 159s 631ms/step - accuracy: 0.7784 - loss: 0.4735 - val_accuracy: 0.7904 - val_loss: 0.4586
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 162s 644ms/step - accuracy: 0.7804 - loss: 0.4539 - val_accuracy: 0.7998 - val_loss: 0.4412
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 161s 641ms/step - accuracy: 0.7999 - loss: 0.4270 - val_accuracy: 0.7988 - val_loss: 0.4314
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 162s 647ms/step - accuracy: 0.8120 -

In [17]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy: ", test_acc)
print("Test Loss: ", test_loss)

64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 134ms/step - accuracy: 0.7777 - loss: 0.5061 
Test Accuracy:  0.7859614491462708
Test Loss:  0.4904727339744568


In [18]:
def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    
    if prediction[0][0] > 0.5:
        print("It's a Dog!")
    else:
        print("It's a Cat!")

# Test on a new image
predict_image('example/pexels-pixabay-45201.jpg', model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
It's a Cat!


In [19]:
model.save('cat_vs_dog_model.h5')